In [20]:
import pandas as pd

# Import dataset
services = pd.read_csv('Aurora_Office_Fase_2_-_all_versions_-_False_-_2023-09-20-21-01-20.csv', sep=';', index_col = False)

# rename variables
# Are still missing the ones related to schedule
newColumns = {'_coordenadas_latitude': 'lat', 
              '_coordenadas_longitude' : 'long',  
              'pais' : 'ubicacion' , 
              'punto_reporte' : 'lugar_enc', 
              'ubicacion' : 'punto_ayud', 
              'cualotropunto' : 'punto_ayud_otro', 
              'infraestructura' : 'cuenta_con' , 
              'sexo' : 'identifica', 
              'organizacionpertence' : 'org_pert', 
              'infraestructuradiscapacidad' : 'serv_disca' , 
              'servicios' : 'serv_tipo'}
services_carto = services.rename(columns=newColumns)

c:\Users\User\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


In [21]:
# serv_tipo id the variable to test (separating by pipe symbol and categorized with number)
# asesorialegal : 1
# alimentacionynutricion : 2
# saludprimerosauxiliosyatencionmedica: 3
# aguapotable: 4
# saneamiento: 5
# higiene: 6
# educacion: 7
# alojamientotemporal: 8
# serviciosdeproteccion : 9
# acompañamientoayudapsicosocial: 10
# transportehumanitario: 11
# restablecimientodecontactofamiliar : 12
# transferencias: 13
# otro: 14
#
services_dict = {
  1 :"asesorialegal",
  2 :"alimentacionynutricion",
  3 :"saludprimerosauxiliosyatencionmedica",
  4 :"aguapotable",
  5 :"saneamiento",
  6 :"higiene",
  7 :"educacion",
  8 :"alojamientotemporal",
  9 :"serviciosdeproteccion",
  10 :"acompañamientoayudapsicosocial",
  11 :"transportehumanitario",
  12 :"restablecimientodecontactofamiliar",
  13:"transferencias",
  14:"Otro",
}

reversed_services_dict = dict( [( x[1],x[0])  for x in services_dict.items()])

def codifyServices(x: str, services_dict: dict[str, int]):
    raw_values = x.split(" ")
    output = []
    for value in raw_values:
        try:      
          codedValue = services_dict[value]
          output.append(str(codedValue) )
        except Exception as e:
            output.append("14")
            
    return "|".join(output)

services_carto["serv_tipo"] = services_carto["serv_tipo"].apply(lambda x: codifyServices(x, reversed_services_dict))

In [22]:
services_carto["serv_tipo"]

0    10|4|7|14
1        10|14
2         12|3
3      10|8|14
Name: serv_tipo, dtype: object